In [ ]:
#installs and imports
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import KFold

In [ ]:
db = datasets.load_diabetes(return_X_y = False, as_frame = False, scaled = True)
print(db.keys())

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])


# Question 1: Preprocessing

## 1.1

Divide the dataset into 2 parts: $D_{tr}$ (training set), and $D_{te}$ (testing set) by randomly placing $80\%$ of the data into $D_{tr}$ and $20\%$ in $D_{te}$. For this, use train_test_split() from sklearn with a random state of $0$


In [ ]:
#use train_test_split
D = db.data
Y = db.target
D_tr, D_te, Y_tr, Y_te = train_test_split(D, Y, test_size = 0.2, random_state = 0)

#Turn D_tr and D_te into dataframes
#D_tr = pd.DataFrame(data = D_tr, columns = db['feature_names'])
#D_tr['target'] = Y_tr

#D_te = pd.DataFrame(data = D_te, columns = db['feature_names'])
#D_te['target'] = Y_te3

In [ ]:
#check size of training set and testing set to ensure accuracy
print('D_tr shape:', D_tr.shape)
print('D_te shape:', D_te.shape)

D_tr shape: (353, 10)
D_te shape: (89, 10)


# Question 2: Linear Regression

## 2.1

Divide $D_{tr}$ further into 2 different sets: $D_{train}$ (training set) and $D_{val}$ (validation set). Place $80\%$ data in $D_{train}$ and remaining in $D_{val}$. Again use random state $0$.

In [ ]:
#use train_test_split
D_train, D_val, Y_train, Y_val = train_test_split(D_tr, Y_tr, test_size = 0.2, random_state = 0)

#check size of arrays to ensure accuracy
print('D_train shape:', D_train.shape)
print('D_val shape:', D_val.shape)

D_train shape: (282, 10)
D_val shape: (71, 10)


## 2.2

Fit the following 4 models separately on $D_{train}$.

(a) $pred = \beta_0 + \beta_1 \cdot bmi + \beta_2 \cdot bp$

(b) $pred = \beta_0 + \beta_1 \cdot bmi + \beta_2 \cdot s5$

(c) $pred = \beta_0 + \beta_1 \cdot bp + \beta_2 \cdot s5$

(d) $pred = \beta_0 + \beta_1 \cdot bmi + \beta_2 \cdot bp + \beta_3 \cdot s5$


and print the $R^2$ value for each of these models on both $D_{train}$ and $D_{val}$.

In [ ]:
#Solve using other methods (not preexisting trained sklearn model)
#concatenate the features for the models
D_a = np.concatenate((np.ones(len(D_train)).reshape(-1,1), D_train[:,2].reshape(-1,1), D_train[:,3].reshape(-1,1)), axis = 1)
D_a_val = np.concatenate((np.ones(len(D_val)).reshape(-1,1), D_val[:,2].reshape(-1,1), D_val[:,3].reshape(-1,1)), axis = 1)

D_b = np.concatenate((np.ones(len(D_train)).reshape(-1,1), D_train[:,2].reshape(-1,1), D_train[:,8].reshape(-1,1)), axis = 1)
D_b_val = np.concatenate((np.ones(len(D_val)).reshape(-1,1), D_val[:,2].reshape(-1,1), D_val[:,8].reshape(-1,1)), axis = 1)

D_c = np.concatenate((np.ones(len(D_train)).reshape(-1,1), D_train[:,3].reshape(-1,1), D_train[:,8].reshape(-1,1)), axis = 1)
D_c_val = np.concatenate((np.ones(len(D_val)).reshape(-1,1), D_val[:,3].reshape(-1,1), D_val[:,8].reshape(-1,1)), axis = 1)

D_d = np.concatenate((np.ones(len(D_train)).reshape(-1,1), D_train[:,2].reshape(-1,1), D_train[:,3].reshape(-1,1), D_train[:,8].reshape(-1,1)), axis = 1)
D_d_val = np.concatenate((np.ones(len(D_val)).reshape(-1,1), D_val[:,2].reshape(-1,1), D_val[:,3].reshape(-1,1), D_val[:,8].reshape(-1,1)), axis = 1)

#calculate coeffecients (betas)
def coeffecients(X, Y):
  XtX = np.dot(X.T, X)
  XtY = np.dot(X.T, Y)
  beta = np.linalg.solve(XtX, XtY)
  return beta

#calculate R^2 score
def find_r_squared(Y_true, Y_pred):
  mean_Y_true = np.mean(Y_true)
  sst = np.sum((Y_true - mean_Y_true) ** 2) #total sum squares
  ssr = np.sum((Y_true - Y_pred) ** 2) #sum-squared regression
  r2 = 1 - (ssr / sst)
  return r2

#define features and results
features = [('(a)', D_a), ('(b)', D_b), ('(c)', D_c), ('(d)', D_d)]
results = {}

#create loop for results
for model_name, D_model in features:
  beta_model = coeffecients(D_model, Y_train)
  pred_train = np.dot(D_model, beta_model)
  pred_val = np.dot(D_a_val if model_name == '(a)' else
                    D_b_val if model_name == '(b)' else
                    D_c_val if model_name == '(c)' else
                    D_d_val, beta_model)
  r2_train = find_r_squared(Y_train, pred_train)
  r2_val = find_r_squared(Y_val, pred_val)
  results[model_name] = {'beta': beta_model, 'r2_train': r2_train, 'r2_val': r2_val}

#print results
for model_name, result in results.items():
  print(f"{model_name} R^2 on D_train: {result['r2_train']:.6f}")
  print(f"{model_name} R^2 on D_val: {result['r2_val']:.6f}\n")

(a) R^2 on D_train: 0.413425
(a) R^2 on D_val: 0.379782

(b) R^2 on D_train: 0.466246
(b) R^2 on D_val: 0.575697

(c) R^2 on D_train: 0.415429
(c) R^2 on D_val: 0.257030

(d) R^2 on D_train: 0.500763
(d) R^2 on D_val: 0.489141



## 2.3

Choose the best model based on your analysis from previous part and fit that model on $D_{tr}$ and and display the $R^2$ on $D_{tr}$ and $D_{te}$.

Based on the results above, I find that the best fit model is that which has the highest R^2 value for the validation set. I also want to avoid models where the R^2 for the training set is higher than the validation set, because that indicates to me that the model is overfitting. Based on this, I immediately see that all of the models, except for (b), are potentially overfitting. Model (b) also has the highest R^2 value for the validation set, so I conclude that model (b) is the best model.

In [ ]:
D_b_tr = np.concatenate((np.ones(len(D_tr)).reshape(-1,1), D_tr[:,2].reshape(-1,1), D_tr[:,8].reshape(-1,1)), axis = 1)
D_b_te = np.concatenate((np.ones(len(D_te)).reshape(-1,1), D_te[:,2].reshape(-1,1), D_te[:,8].reshape(-1,1)), axis = 1)

beta = coeffecients(D_b_tr, Y_tr)

pred_tr = np.dot(D_b_tr, beta)
pred_te = np.dot(D_b_te, beta)
r2_tr = find_r_squared(Y_tr, pred_tr)
r2_te = find_r_squared(Y_te, pred_te)

print("(b) R^2 on D_tr: {:.6f}".format(r2_tr))
print("(b) R^2 on D_te: {:.6f}".format(r2_te))

(b) R^2 on D_tr: 0.496123
(b) R^2 on D_te: 0.283540


## 2.4

Explain the problems with the model selection approach you implemented in part 1, 2, and 3 of this question.

Interestingly, with each step of this question, I learned a lot about splitting datasets, using the test_train_split() function. When implementing the models a, b, c, and d, when performing this on the D_train and D_val sets, model b fit the data best, having a higher R^2 value for the D_val vs the D_train, which is a sign of a good fitting model. When implemented on D_tr and D_te however, the original data that was split, the model performed significantly worse for D_te.


# Question 3: Ridge Regression

For the linear model:

> $pred = \beta_0 + \beta_1 \cdot bmi + \beta_2 \cdot bp + \beta_3 \cdot s5$

with $X$ and $Y$ containing the appropriate data, we now wish to fit a ridge regression model which has the following optimal weights:

> $\hat\beta^{ridge} = (X^TX+n\lambda I)^{-1}X^TY$

Hence for fixed $X$ and $Y$ matrices, the problem of fitting a ridge regression model boils down to finding the right $\lambda$. In this regard:

## 3.1

Use Generalized Cross Validation (CV) metric to compute and display the best $\lambda$ on $D_{tr}$.

In [ ]:
#utilizing model (d) on D_tr
D_d_tr = np.concatenate((np.ones(len(D_tr)).reshape(-1,1), D_tr[:,2].reshape(-1,1), D_tr[:,3].reshape(-1,1), D_tr[:,8].reshape(-1,1)), axis = 1)

#GCV Function provided in lectures
def GCV(lam,X,y):
    beta = np.linalg.inv(X.T.dot(X) + X.shape[0]*lam*np.eye(X.shape[1])).dot(X.T.dot(y))
    yhat = X.dot(beta)
    H = X.dot(np.linalg.inv(X.T.dot(X)).dot(X.T))
    h = np.mean(np.diag(H))
    s = 0
    for i in range(len(y)):
        term = (y[i] - yhat[i])/(1 - h)
        s = s + term**2
    return s/len(y)

lam = np.linspace(1.0e-10, 10, 100)
gcv = [GCV(i, D_d_tr, Y_tr) for i in lam]

#optimize lambda for GCV
def optimize_lamGCV(X,y,lam):
    args = (X,y)
    bnds = [(1.0e-12, None)]
    lamb = [lam]
    lam = minimize(GCV,lamb,args,bounds=bnds,method='SLSQP')
    return lam

#show lambda
lam = optimize_lamGCV(D_d_tr,Y_tr,lam = 0.1)
lam

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3073.5000572849535
       x: [ 1.000e-12]
     nit: 2
     jac: [ 3.278e+00]
    nfev: 4
    njev: 2

In [ ]:
lam.x

array([1.00000563e-12])

In [ ]:
#print results
best_lambda = lam.x
print('The optimal lambda value for D_tr is:', best_lambda)

The optimal lambda value for D_tr is: [1.00000563e-12]


## 3.2

Using the best obtained $\lambda$, compute and display the corresponding $\hat \beta^{ridge}$ on $D_{tr}$.

In [ ]:
n = len(Y_tr)

#determine beta coeff using ridge regression
beta_ridge = np.linalg.inv(D_d_tr.T.dot(D_d_tr) + n*(np.exp(-12))*np.eye(D_d_tr.shape[1])).dot(D_d_tr.T.dot(Y_tr))

#print results
coeff_titles = ['β0 ridge', 'β1 ridge', 'β2 ridge', 'β3 ridge']

#print coeffecients with titles
for title, coeffecient in zip (coeff_titles, beta_ridge):
  print(f'{title}: {coeffecient}')

β0 ridge: 152.17875866243705
β1 ridge: 636.8218182537266
β2 ridge: 224.00347504875617
β3 ridge: 559.929329152485


## 3.3

Using $\hat \beta^{ridge}$ from previous part, compute and display $R^2$ on $D_{te}$.

In [ ]:
D_d_te= np.concatenate((np.ones(len(D_te)).reshape(-1,1),D_te[:,2].reshape(-1,1), D_te[:,3].reshape(-1,1), D_te[:,8].reshape(-1,1)), axis = 1)

#Continuing from part 3.2
Y_ridge = D_d_te.dot(beta_ridge)
r2 = find_r_squared(Y_te, Y_ridge)

#print result
print(f'R^2 on D_te: {r2}')

R^2 on D_te: 0.33378780900313443


# Question 4: Lasso Regression

Again for the linear model:

> $pred = \beta_0 + \beta_1 \cdot bmi + \beta_2 \cdot bp + \beta_3 \cdot s5$

with $X$ and $Y$ containing the appropriate data, we now wish to fit a lasso regression model. Here again the objective of fitting a lasso regression model boils down to finding the right $\lambda$. In this regard:

## 4.1

On $D_{tr}$, Use LassoCV from sklearn (again use a random state of 0) to do a K-fold cross validation base selection of best $\lambda$. Display the best $\lambda$ you obtain. Use $K = 5$ here.

In [ ]:
alphas = np.logspace(1.0e-10, 10, 100)
lasso_cv = LassoCV(alphas=alphas, cv=KFold(n_splits=5))

#fit lassocv to model D_tr
lasso_cv.fit(D_d_tr, Y_tr)

#print best lambda
best_alpha = lasso_cv.alpha_
print("The optimal lambda for D_tr is:", best_alpha)

The optimal lambda for D_tr is: 1.0000000002302585


## 4.2

Using the best obtained $\lambda$ in part 1 above, compute the predictions on $D_{te}$ and display the corresponding $R^2$ on $D_{te}$.

In [ ]:
#fit lasso model based on best_alpha
lasso = Lasso(alpha = best_alpha)
lasso.fit(D_d_tr, Y_tr)

#compute predictions and find r-squared
pred = lasso.predict(D_d_te)
r2 = find_r_squared(Y_te, pred)

#Print results
print("R^2 on D_te:", r2)

R^2 on D_te: 0.2613246825079295


## 4.3

Comment on the relative performance of linear regression, ridge regression and lasso regression for this problem.

For linear regression, I think there's interesting results in the rigidity of the models based on the training data vs the validation data. When a model works really well for one set of train_test_split data from the same dataset, when applied to another set of that data from another perspective, the model does not perform as well.

For ridge regression, the model was more difficult to interpret and create, from my personal view. However, tbe R^2 value on the testing data was higher than that of the Lasso Regression model, leading me to believe that although Lasso Regression was easier to develop, the Ridge Regression model seems to fit the particular linear function better between the models. Linear regression can give us a peak at what we are directly looking at, but when applied to the testing data, the R^2 value lowers, showing a disconnect between those steps.

For this model I would most likely choose Ridge Regression as the best model.